# Simulate R-map with various sample size and (combinations of) circular/clocklike

In [1]:
import os
import sys
sys.path.insert(0, "/Users/hyl/Desktop/W21/project/Erdbeermet")
from src.erdbeermet.simulation import simulate

In [3]:
numRep = 25000
for n in [6, 10, 15, 20, 25]:
    for i in range(1, 1+numRep):
        
        if not os.path.isdir(f'./simulations/N{n}/default'):
            os.mkdir(f'./simulations/N{n}/default')

        if not os.path.isdir(f'./simulations/N{n}/circ'):
            os.mkdir(f'./simulations/N{n}/circ')
        
        if not os.path.isdir(f'./simulations/N{n}/clock'):
            os.mkdir(f'./simulations/N{n}/clock')

        if not os.path.isdir(f'./simulations/N{n}/circ_clock'):
            os.mkdir(f'./simulations/N{n}/circ_clock')

        scenario = simulate(n, branching_prob=0.0, circular=False, clocklike=False)
        scenario.write_history(f'./simulations/N{n}/default/hist_rep{i}.txt')

        scenario = simulate(n, branching_prob=0.0, circular=True, clocklike=False)
        scenario.write_history(f'./simulations/N{n}/circ/hist_rep{i}.txt')

        scenario = simulate(n, branching_prob=0.0, circular=False, clocklike=True)
        scenario.write_history(f'./simulations/N{n}/clock/hist_rep{i}.txt')

        scenario = simulate(n, branching_prob=0.0, circular=True, clocklike=True)
        scenario.write_history(f'./simulations/N{n}/circ_clock/hist_rep{i}.txt')




In [2]:
import multiprocessing as mp
def multi_run(fun, prms, processes = 4, output=False):
    """Implementation of running in Parallel.
    fun: Function
    prms: The Parameter Files
    processes: How many Processes to use"""
    if output:
        print(f"Running {len(prms)} total jobs; {processes} in parallel.")
    
    if len(prms)>1:
        if output:
            print("Starting Pool of multiple workers...")    
        with mp.Pool(processes = processes) as pool:
            results = pool.starmap(fun, prms)
    elif len(prms)==1:
        if output:
            print("Running single process...")
        results = fun(*prms[0])
    else:
        raise RuntimeWarning("Nothing to run! Please check input.")
    return results

# WP2 Original Algorithm

In [ ]:
from src.erdbeermet.simulation import load
from src.erdbeermet.recognition import recognize, recognize_and_compare
from tqdm import tqdm
import pickle
import os

# prms = [[load(f'./simulations/N20/default/hist_rep{r}.txt').D, True, False] for r in range(1,51)]
# multi_run(recognize, prms, processes=4)

for mode in ['default', 'clock', 'circ', 'circ_clock']:
    runtime = []
    common_triple = []
    failed_index = []
    failed_tree = []
    init_leaves_recovered = []

    basepath = f'./simulations/N8/{mode}'

    outdir = f'{basepath}/results/wp2'
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    for r in tqdm(range(1,25000+1)):
        scenario = load(f'{basepath}/hist_rep{r}.txt')
        try:
            return_val = recognize_and_compare(scenario, first_candidate_only=True, print_info=False)
        except:
            print(f'an error occured for {r}')
            failed_index.append(r)
            failed_tree.append(None)
            continue
        if len(return_val) == 2:
            print('recognition failed')
            recognition_tree, t = return_val
            failed_index.append(r)
            failed_tree.append(recognition_tree)
        else:
            recognition_tree, t, count, init_quad_recovered = return_val
            runtime.append(t)
            common_triple.append(count)
            init_leaves_recovered.append(init_quad_recovered)

    # dump the recognition results
    pickle.dump(failed_index, open(f'{outdir}/failed_index', 'wb'))
    pickle.dump(failed_tree, open(f'{outdir}/failed_tree', 'wb'))
    pickle.dump(runtime, open(f'{outdir}/runtime', 'wb'))
    pickle.dump(common_triple, open(f'{outdir}/common_triple', 'wb'))
    pickle.dump(init_leaves_recovered, open(f'{outdir}/init_leaves_recovered', 'wb'))



# WP3: Blocked Leaf

In [2]:
from src.erdbeermet.simulation import load
from src.erdbeermet.recognition import recognize, recognize_and_compare
from tqdm import tqdm
import pickle
import os

# prms = [[load(f'./simulations/N20/default/hist_rep{r}.txt').D, True, False] for r in range(1,51)]
# multi_run(recognize, prms, processes=4)

for mode in ['default', 'clock', 'circ', 'circ_clock']:
    runtime = []
    common_triple = []
    failed_index = []
    failed_tree = []
    init_leaves_recovered = []

    basepath = f'./simulations/N10/{mode}'

    outdir = f'{basepath}/results/WP3.2_3leaves'
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    for r in tqdm(range(1,1000+1)):
        scenario = load(f'{basepath}/hist_rep{r}.txt')
        try:
            return_val = recognize_and_compare(scenario, use_unknown_core_leaves=3, first_candidate_only=True, print_info=False)
        except:
            print(f'an error occured for {r}')
            failed_index.append(r)
            failed_tree.append(None)
            continue
        if len(return_val) == 2:
            print('recognition failed')
            recognition_tree, t = return_val
            failed_index.append(r)
            failed_tree.append(recognition_tree)
        else:
            recognition_tree, t, count, init_quad_recovered = return_val
            runtime.append(t)
            common_triple.append(count)
            init_leaves_recovered.append(init_quad_recovered)

    # dump the recognition results
    pickle.dump(failed_index, open(f'{outdir}/failed_index', 'wb'))
    pickle.dump(failed_tree, open(f'{outdir}/failed_tree', 'wb'))
    pickle.dump(runtime, open(f'{outdir}/runtime', 'wb'))
    pickle.dump(common_triple, open(f'{outdir}/common_triple', 'wb'))
    pickle.dump(init_leaves_recovered, open(f'{outdir}/init_leaves_recovered', 'wb'))








  0%|          | 2/1000 [00:02<19:16,  1.16s/it]

an error occured for 2


  1%|          | 7/1000 [00:14<43:54,  2.65s/it]

an error occured for 8

  1%|          | 8/1000 [00:15<32:30,  1.97s/it]

# WP4: Smallest Spike

In [ ]:
from src.erdbeermet.simulation import load
from src.erdbeermet.recognition import recognize, recognize_and_compare
from tqdm import tqdm
import pickle
import os

for mode in ['default', 'clock', 'circ', 'circ_clock']:
    runtime = []
    common_triple = []
    failed_index = []
    failed_tree = []
    init_leaves_recovered = []

    basepath = f'./simulations/N10/{mode}'

    outdir = f'{basepath}/results/wp4'
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    for r in tqdm(range(1,25000+1)):
        scenario = load(f'{basepath}/hist_rep{r}.txt')
        try:
            return_val = recognize_and_compare(scenario, use_spikes=True, first_candidate_only=True, print_info=False)
        except:
            print(f'an error occured for {r}')
            failed_index.append(r)
            failed_tree.append(None)
            continue
        if len(return_val) == 2:
            print('recognition failed')
            recognition_tree, t = return_val
            failed_index.append(r)
            failed_tree.append(recognition_tree)
        else:
            recognition_tree, t, count, init_quad_recovered = return_val
            runtime.append(t)
            common_triple.append(count)
            init_leaves_recovered.append(init_quad_recovered)

    # dump the recognition results
    pickle.dump(failed_index, open(f'{outdir}/failed_index', 'wb'))
    pickle.dump(failed_tree, open(f'{outdir}/failed_tree', 'wb'))
    pickle.dump(runtime, open(f'{outdir}/runtime', 'wb'))
    pickle.dump(common_triple, open(f'{outdir}/common_triple', 'wb'))
    pickle.dump(init_leaves_recovered, open(f'{outdir}/init_leaves_recovered', 'wb'))



